<a href="https://colab.research.google.com/github/jiayiwang5/Google-Colab/blob/master/zh_chatbot_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pickle
import operator
main_path = '/content/drive/My Drive/Colab Notebooks/'
question = np.load(main_path + 'middle_data/' + 'pad_question.npy')
answer = np.load(main_path + 'middle_data/' + 'pad_answer.npy')
answer_o = np.load(main_path + 'middle_data/' + 'answer_o.npy')
with open(main_path + 'middle_data/' + 'vocab_bag.pkl', 'rb') as f:
    words = pickle.load(f)
with open(main_path + 'middle_data/' + 'pad_word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open(main_path + 'middle_data/' + 'pad_index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)
vocab_size = len(word_to_index) + 1
maxLen=20
def get_file_list(file_path):
    dir_list = os.listdir(file_path)
    if not dir_list:
        return
    else:
        dir_list = sorted(dir_list, key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
    return dir_list

In [4]:
from keras.preprocessing import sequence
def generate_train(batch_size):
    print('\n*********************************generate_train()*********************************')
    steps=0
    question_ = question
    answer_ = answer
    while True:
        batch_answer_o = answer_o[steps:steps+batch_size]
        batch_question = question_[steps:steps+batch_size]
        batch_answer = answer_[steps:steps+batch_size]
        outs = np.zeros([batch_size, maxLen, vocab_size], dtype='float32')
        for pos, i in enumerate(batch_answer_o):
            for pos_, j in enumerate(i):
                if pos_ > 20:
                    print(i)
                if pos_ > 0:
                    outs[pos, pos_-1, j] = 1 # one-hot
        yield [batch_question, batch_answer], outs
        steps += batch_size
        if steps == 100000:
            steps = 0

Using TensorFlow backend.


In [0]:
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed, Bidirectional, Dropout, Concatenate, RepeatVector, Activation, Dot
from keras.layers import concatenate, dot                    
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, TensorBoard,ReduceLROnPlateau
from keras.initializers import TruncatedNormal
import pydot
import os, re
truncatednormal = TruncatedNormal(mean=0.0, stddev=0.05, seed=5)
embed_layer = Embedding(input_dim=vocab_size, 
                        output_dim=100, 
                        trainable=True,
                        embeddings_initializer= truncatednormal)
embed_layer.build((None,))

LSTM_encoder = Bidirectional(LSTM(256,
                                  return_sequences=True, 
                                  return_state=True, 
                                  activation = 'relu',
                                  dropout=0.25, 
                                  recurrent_dropout=0.1
                             ))
LSTM_propagate = LSTM(512,
                      return_sequences=True,
                      return_state=True,
                      activation='relu',
                      dropout=0.25,
                      recurrent_dropout=0.1
                        )
LSTM_decoder = LSTM(512, 
                    return_sequences=True, 
                    return_state=True, 
                    activation = 'relu',
                    dropout=0.25, 
                    recurrent_dropout=0.1
                   )

#encoder输入 与 decoder输入
input_question = Input(shape=(None, ), dtype='int32', name='input_question')
input_answer = Input(shape=(None, ), dtype='int32', name='input_answer')

input_question_embed = embed_layer(input_question)
input_answer_embed = embed_layer(input_answer)

# encoder_lstm, forward_h, forward_c, backward_h, backward_c = LSTM_encoder(input_question_embed)

# question_h_e = Concatenate()([forward_h, backward_h]) 
# question_c_e = Concatenate()([forward_c, backward_c])

encoder_lstm_, question_h, question_c = LSTM_propagate(input_question_embed)

decoder_lstm, _, _ = LSTM_decoder(input_answer_embed, 
                                  initial_state=[question_h, question_c])

attention = dot([decoder_lstm, encoder_lstm_], axes=[2, 2])
attention = Activation('softmax')(attention)
context = dot([attention, encoder_lstm_], axes=[2,1])
decoder_combined_context = concatenate([context, decoder_lstm])

dense1 = TimeDistributed(Dense(256, activation='tanh'))
dense2 = TimeDistributed(Dense(vocab_size, activation='softmax'))

# output = dense1(decoder_combined_context)
output = dense2(decoder_lstm)

model = Model([input_question, input_answer], output)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', 
              metrics=['accuracy'])

filepath = main_path + "modles/W3-" + "-{epoch:3d}-{loss:.4f}-.h5"
checkpoint = ModelCheckpoint(filepath,
                             monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min',
                             period=1,
                             save_weights_only=True
                             )
reduce_lr = ReduceLROnPlateau(monitor='loss', 
                              factor=0.2, 
                              patience=5, 
                              verbose=1, 
                              mode='min', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=0
                              )
tensorboard = TensorBoard(log_dir=main_path + 'logs', 
                          histogram_freq=0, 
                          batch_size=100, 
                          write_graph=True, 
                          write_grads=True, 
                          write_images=True, 
                          embeddings_freq=0, 
                          embeddings_layer_names=None, 
                          embeddings_metadata=None, 
                          embeddings_data=None, 
                          update_freq='epoch'
                         )
callbacks_list = [checkpoint, reduce_lr, tensorboard]

initial_epoch_=0
file_list = os.listdir(main_path + 'modles/')
if len(file_list) > 0:
  epoch_list = get_file_list(main_path + 'modles/')
  epoch_last = epoch_list[-1]
  model.load_weights(main_path + 'modles/' + epoch_last)
  print("**********checkpoint_loaded: ", epoch_last)
  initial_epoch_ = int(epoch_last.split('-')[2]) - 1
  print('**********Begin from epoch: ', str(initial_epoch_))

model.fit_generator(generate_train(batch_size=100), 
                    steps_per_epoch=1000, # (total samples) / batch_size 90000/100 = 900
                    epochs=300, 
                    verbose=1, 
                    callbacks=callbacks_list, 
#                     validation_data=generate_test(batch_size=100), 
#                     validation_steps=200, # 10000/100 = 100
                    class_weight=None, 
                    max_queue_size=5, 
                    workers=1, 
                    use_multiprocessing=False, 
                    shuffle=False, 
                    initial_epoch=initial_epoch_
                    )

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
**********checkpoint_loaded:  W3-- 33-0.4161-.h5
**********Begin from epoch:  32
Instructions for updating:
Use tf.cast instead.
Epoch 33/300

*********************************generate_train()*********************************
 430/1000 [===========>..................] - ETA: 5:47 - loss: 0.4134 - acc: 0.2728

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pwd
!ls "/content/drive/My Drive/Colab Notebooks/modles/"

/content
'W3-- 10-0.6828-.h5'  'W3-- 20-0.5233-.h5'  'W3-- 30-0.4364-.h5'
'W3-- 11-0.6598-.h5'  'W3-- 21-0.5129-.h5'  'W3-- 31-0.4288-.h5'
'W3--  1-1.6841-.h5'  'W3--  2-1.2278-.h5'  'W3--  3-1.0645-.h5'
'W3-- 12-0.6385-.h5'  'W3-- 22-0.5029-.h5'  'W3-- 32-0.4230-.h5'
'W3-- 13-0.6196-.h5'  'W3-- 23-0.4931-.h5'  'W3-- 33-0.4161-.h5'
'W3-- 14-0.6012-.h5'  'W3-- 24-0.4843-.h5'  'W3--  4-0.9612-.h5'
'W3-- 15-0.5857-.h5'  'W3-- 25-0.4755-.h5'  'W3--  5-0.8858-.h5'
'W3-- 16-0.5705-.h5'  'W3-- 26-0.4671-.h5'  'W3--  6-0.8278-.h5'
'W3-- 17-0.5577-.h5'  'W3-- 27-0.4592-.h5'  'W3--  7-0.7799-.h5'
'W3-- 18-0.5458-.h5'  'W3-- 28-0.4511-.h5'  'W3--  8-0.7423-.h5'
'W3-- 19-0.5339-.h5'  'W3-- 29-0.4432-.h5'  'W3--  9-0.7097-.h5'
